In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.4"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"
#r "nuget: Microsoft.Extensions.AI, 9.8.0"
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages Azure.AI.Agents.Persistent, 1.2.0-beta.4 Azure.Identity, 1.15.0 DotNetEnv, 3.1.1 Microsoft.Extensions.AI, 9.8.0 System.Linq.Async, 6.0.3

In [2]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.Orchestration\bin\Debug\net9.0\Microsoft.Agents.Orchestration.dll"

In [3]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Extensions.AI.Agents.AzureAI\bin\Debug\net9.0\Microsoft.Extensions.AI.Agents.AzureAI.dll"

In [4]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Extensions.AI.Agents\bin\Debug\net9.0\Microsoft.Extensions.AI.Agents.dll"

In [5]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Threading.Tasks;
using Azure.AI.Agents.Persistent;
using Azure.Identity;
using Microsoft.Extensions.AI;
using Microsoft.Extensions.AI.Agents;
using DotNetEnv;

In [6]:
// Load environment variables
Env.Load("../../../.env");

var azure_foundry_endpoint = Environment.GetEnvironmentVariable("FOUNDRY_PROJECT_ENDPOINT") ?? throw new InvalidOperationException("FOUNDRY_PROJECT_ENDPOINT is not set.");
var azure_foundry_model_id = "gpt-4.1";

var bing_conn_id = Environment.GetEnvironmentVariable("BING_CONNECTION_NAME");

In [7]:
var bingGroundingConfig = new BingGroundingSearchConfiguration(bing_conn_id);

BingGroundingToolDefinition bingGroundingTool = new(
    new BingGroundingSearchToolParameters(
        [bingGroundingConfig]
    )
);

In [8]:
// Create the persistent agents client
var persistentAgentsClient = new PersistentAgentsClient(azure_foundry_endpoint, new AzureCliCredential());

In [9]:
const string EvangelistInstructions = @"
You are a technical evangelist, writing a tutorial.
1. The generated content format is markdown
2. Generate a title based on the content
3. Each knowledge point can be accessed through related links to obtain knowledge points and content
4. The content should not include code examples
5. The word count should be no less than 1000 words, and each point needs to be integrated in detail.
";

const string ContentReviewerInstructions = @"
You are a content reviewer for a publishing company. You need to check whether the content meets the following requirements:

1. If code samples are included, they must include both dotNET and Python code. If there are no code samples, skip this requirement.

If the above requirements are met, return yes; otherwise, return no.
";

const string PublisherInstructions = @"
You are the content publisher and save the content. Saved file's name is marked with current date and time, such as yearmonthdayhourminsec. Note that if it is 1-9, you need to add 0, such as 01.saved the markdown file such as 20240101123045.md. Then Save this content in this file
";

In [10]:
// Create the three specialized agents
var evangelistMetadata = await persistentAgentsClient.Administration.CreateAgentAsync(
    model: azure_foundry_model_id,
    name: "Evangelist",
    instructions: EvangelistInstructions,
    tools: [bingGroundingTool]
);

var contentReviewerMetadata = await persistentAgentsClient.Administration.CreateAgentAsync(
     model: azure_foundry_model_id,
     name: "ContentReviewer", 
     instructions: ContentReviewerInstructions
);

var publisherMetadata = await persistentAgentsClient.Administration.CreateAgentAsync(
    model: azure_foundry_model_id,
    name: "Publisher",
    instructions: PublisherInstructions,
    tools: [new CodeInterpreterToolDefinition()]
);

In [12]:
string evangelist_agentId = evangelistMetadata.Value.Id;
string contentReviewer_agentId = contentReviewerMetadata.Value.Id;
string publisher_agentId = publisherMetadata.Value.Id;

In [13]:
AIAgent evangelistagent = await persistentAgentsClient.GetAIAgentAsync(evangelist_agentId);
AIAgent contentRevieweragent = await persistentAgentsClient.GetAIAgentAsync(contentReviewer_agentId);
AIAgent publisheragent = await persistentAgentsClient.GetAIAgentAsync(publisher_agentId);

In [14]:
AgentThread evangelistthread = evangelistagent.GetNewThread();
AgentThread contentReviewerthread = contentRevieweragent.GetNewThread();
AgentThread publisherthread = publisheragent.GetNewThread();


In [15]:
// Method to get file contents from local or remote sources
async Task<string> GetFileContents(string filePath)
{
    try
    {
        if (File.Exists(filePath))
        {
            Console.WriteLine($"📁 Reading local file: {filePath}");
            return await File.ReadAllTextAsync(filePath);
        }
        else
        {
            throw new FileNotFoundException($"File not found: {filePath}");
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error reading file {filePath}: {ex.Message}");
        return $"Error reading file: {ex.Message}";
    }
}


In [16]:
// Method to get multiple file contents at once
async Task<Dictionary<string, string>> GetMultipleFileContents(params string[] filePaths)
{
    var results = new Dictionary<string, string>();
    
    Console.WriteLine($"📚 Processing {filePaths.Length} files...");
    
    foreach (string filePath in filePaths)
    {
        try
        {
            string content = await GetFileContents(filePath);
            results[filePath] = content;
            Console.WriteLine($"✅ Successfully loaded: {Path.GetFileName(filePath)}");
        }
        catch (Exception ex)
        {
            results[filePath] = $"Error: {ex.Message}";
            Console.WriteLine($"❌ Failed to load: {filePath}");
        }
    }
    
    return results;
}

In [17]:
string[] files= [
    "C:\\Users\\kinfeylo\\Documents\\Agent\\AgentframeworkInAction\\01.AgentFoundation\\agent-framework-workflow-architecture.md",
    "C:\\Users\\kinfeylo\\Documents\\Agent\\AgentframeworkInAction\\01.AgentFoundation\\README.md"
];

In [18]:
var content = await GetMultipleFileContents(files);

In [19]:
ChatMessage userMessage = new ChatMessage(ChatRole.User, [
	new TextContent(content["C:\\Users\\kinfeylo\\Documents\\Agent\\AgentframeworkInAction\\01.AgentFoundation\\README.md"])
]);

In [20]:
string content = "";

In [21]:
string responseContent = "";
await foreach (var update in evangelistagent.RunStreamingAsync(userMessage, evangelistthread))
{
    responseContent += update;
    //Console.Write(update);
}

In [22]:
ChatMessage userMessage = new ChatMessage(ChatRole.User, [
	new TextContent(responseContent)
]);

In [ ]:
await foreach (var update in publisheragent.RunStreamingAsync(userMessage, publisherthread))
{
    //responseContent += update;
    Console.Write(update);

}


The file as 20250912150352.md

Now saving content to this file. content has been saved successfully in the file: **509**.

You can download your Markdown file from the link below:

[Download 20250912150352](sandbox:/mnt/data/20250912150352.md)

In [24]:
publisherthread.ConversationId

thread_46S343a2M8lCvXUFEs6QF6nK

In [25]:

using Microsoft.Extensions.Configuration;
using System.Diagnostics;

In [26]:
using Azure;

Pageable<PersistentThreadMessage> messages = persistentAgentsClient.Messages.GetMessages(
    threadId: publisherthread.ConversationId
);

In [27]:
messages

[ Azure.AI.Agents.Persistent.PersistentThreadMessage, Azure.AI.Agents.Persistent.PersistentThreadMessage, Azure.AI.Agents.Persistent.PersistentThreadMessage ] (values) index value 0 Azure.AI.Agents.Persistent.PersistentThreadMessage Id msg_hSbnomdg7MdRJdMu1hv7rEBD CreatedAt 2025-09-12 15:04:14Z ThreadId thread_46S343a2M8lCvXUFEs6QF6nK Status System.NullReferenceException: Object reference not set to an instance of an object.\r\n at Microsoft.DotNet.Interactive.Formatting.PlainTextSummaryFormatter.Truncate(Object value) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\PlainTextSummaryFormatter.cs:line 85\r\n at Microsoft... IncompleteDetails <null> CompletedAt <null> IncompleteAt <null> Role assistant ContentItems index value 0 Azure.AI.Agents.Persistent.MessageTextContent Text The content has been saved successfully in the file: **20250912150352.md**.

You can download your Markdown file from the link below:

[Download 20250912150352.md](sandbox:/mnt/data/20250912150352.md) Annotations index value 0 Azure.AI.Agents.Persistent.MessageTextFilePathAnnotation FileId assistant-CYC3dsNDepUZKT3NciV5Wd StartIndex 164 EndIndex 199 Text sandbox:/mnt/data/20250912150352.md AssistantId asst_tz6xmTlOUCPKW5ScspYA7E1y RunId run_9lFM8Eyjp1q19fVKApItLPfd Attachments index value 0 Azure.AI.Agents.Persistent.MessageAttachment FileId assistant-CYC3dsNDepUZKT3NciV5Wd DataSource <null> Tools index value 0 {\n "type": "code_interpreter"\n } Length 56 IsEmpty False MediaType <null> Metadata (empty) 1 Azure.AI.Agents.Persistent.PersistentThreadMessage Id msg_SyHN8JQ8Fa7nq8msPJAS46Xd CreatedAt 2025-09-12 15:03:52Z ThreadId thread_46S343a2M8lCvXUFEs6QF6nK Status System.NullReferenceException: Object reference not set to an instance of an object.\r\n at Microsoft.DotNet.Interactive.Formatting.PlainTextSummaryFormatter.Truncate(Object value) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\PlainTextSummaryFormatter.cs:line 85\r\n at Microsoft... IncompleteDetails <null> CompletedAt <null> IncompleteAt <null> Role assistant ContentItems index value 0 Azure.AI.Agents.Persistent.MessageTextContent Text The file will be saved as: 20250912150352.md

Now saving content to this file. Annotations (empty) AssistantId asst_tz6xmTlOUCPKW5ScspYA7E1y RunId run_9lFM8Eyjp1q19fVKApItLPfd Attachments (empty) Metadata (empty) 2 Azure.AI.Agents.Persistent.PersistentThreadMessage Id msg_dDlFLDzBDgvKd6OxxnhQHZA1 CreatedAt 2025-09-12 15:03:49Z ThreadId thread_46S343a2M8lCvXUFEs6QF6nK Status System.NullReferenceException: Object reference not set to an instance of an object.\r\n at Microsoft.DotNet.Interactive.Formatting.PlainTextSummaryFormatter.Truncate(Object value) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\PlainTextSummaryFormatter.cs:line 85\r\n at Microsoft... IncompleteDetails <null> CompletedAt <null> IncompleteAt <null> Role user ContentItems index value 0 Azure.AI.Agents.Persistent.MessageTextContent Text # Comprehensive Introduction to AI Agents and Microsoft’s AI Agent Ecosystem

Artificial Intelligence (AI) agents are at the core of the latest wave of intelligent application development. These agents automate decision-making tasks, interact with humans or other systems, and help organizations leverage machine learning (ML) and automation for greater productivity. Microsoft has pioneered several robust frameworks and cloud services for developing and deploying AI agents, providing developers with industry-grade tools to bring intelligent agents to production. This tutorial offers a detailed overview of the fundamental concepts of AI agents, introduces the Azure AI Foundry Agent Service—a cloud-native platform for agent orchestration—and explores the Microsoft Agent Framework for building custom and extensible agent solutions.

---

## Understanding AI Agents

AI agents represent autonomous entities capable of perceiving their environment, reasoning about their inputs, and acting upon their observations to achieve designa

In [28]:
string GetFileName(string filePath)
{
    try
    {
        // Handle sandbox paths like "sandbox:/mnt/data/filename.md"
        if (filePath.StartsWith("sandbox:"))
        {
            // Remove the sandbox prefix and extract filename
            string cleanPath = filePath.Replace("sandbox:", "");
            return Path.GetFileName(cleanPath);
        }
        
        // Handle regular file paths
        return Path.GetFileName(filePath);
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error extracting filename from {filePath}: {ex.Message}");
        return filePath; // Return original path if extraction fails
    }
}

In [29]:


foreach (PersistentThreadMessage threadMessage in messages)
{
    foreach (MessageContent content in threadMessage.ContentItems)
    {
        switch (content)
        {
            case MessageTextContent textContent:
                if(textContent.Annotations is not null && textContent.Annotations.Count > 0)
                {

                    if(textContent.Annotations[0] is MessageTextFilePathAnnotation pathItem )
                    {
                        Console.WriteLine(pathItem.FileId);

                        BinaryData filecontent =  persistentAgentsClient.Files.GetFileContent(pathItem.FileId);
                        string tempFilePath1 = "c:\\Users\\kinfeylo\\Documents\\Agent\\AgentframeworkInAction\\07.MultiAgents\\code_samples\\dotNET\\outputs\\" + GetFileName(pathItem.Text);
                        File.WriteAllBytes(tempFilePath1, filecontent.ToArray());
                        persistentAgentsClient.Files.DeleteFile(pathItem.FileId);

                        ProcessStartInfo psi1 = new()
                        {
                            FileName = tempFilePath1,
                            UseShellExecute = true
                        };
                        Process.Start(psi1);
                    }
                }
                break;
            default:
                Console.WriteLine($"[{threadMessage.Role}] Unknown content type.");
                break;
        }
    }
}

assistant-CYC3dsNDepUZKT3NciV5Wd


In [30]:
responseContent

# Comprehensive Introduction to AI Agents and Microsoft’s AI Agent Ecosystem

Artificial Intelligence (AI) agents are at the core of the latest wave of intelligent application development. These agents automate decision-making tasks, interact with humans or other systems, and help organizations leverage machine learning (ML) and automation for greater productivity. Microsoft has pioneered several robust frameworks and cloud services for developing and deploying AI agents, providing developers with industry-grade tools to bring intelligent agents to production. This tutorial offers a detailed overview of the fundamental concepts of AI agents, introduces the Azure AI Foundry Agent Service—a cloud-native platform for agent orchestration—and explores the Microsoft Agent Framework for building custom and extensible agent solutions.

---

## Understanding AI Agents

AI agents represent autonomous entities capable of perceiving their environment, reasoning about their inputs, and acting upon 

In [31]:
ChatMessage userMessage = new ChatMessage(ChatRole.User, [
	new TextContent(responseContent)
]);

In [32]:
await foreach (var update in contentRevieweragent.RunStreamingAsync(userMessage, contentReviewerthread))
{
    //responseContent += update;
    Console.Write(update);

}

yes